In [1]:
%matplotlib inline

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import scipy as sp
import seaborn as sns
import warnings

from theano import tensor as tt

warnings.simplefilter(action='ignore', category=FutureWarning)

sns.set(context='notebook', font_scale=1.2, rc={'figure.figsize': (12, 5)})
plt.style.use(['seaborn-colorblind', 'seaborn-darkgrid'])

RANDOM_SEED = 8927
np.random.seed(286)

# Helper function
def stdz(series: pd.Series):
    """Standardize the given pandas Series"""
    return (series - series.mean())/series.std()

### 14E1.
*Add to the following model varying slopes on the predictor $x$:*

$y_{i} \sim Normal(\mu_{i}, \sigma)$

$\mu_{i} = \alpha_{GROUP[i]} + \beta x_{i}$

$\alpha_{GROUP} \sim Normal(\alpha, \sigma_{\alpha})$

$\alpha \sim Normal(0, 10)$

$\beta \sim Normal(0, 1)$

$\sigma \sim HalfCauchy(2)$

$\sigma_{\alpha} \sim HalfCauchy(2)$

Let's do it! To add a varying slope on x,we have to add a dimension to the adaptive prior. This will mean that the $\beta$ parameter becomes the average slope, and then we’ll need a new standard deviation parameter for the slopes and a correlation parameter to estimate the correlation between intercepts and slopes. You can find an annotated example on page 407. Keep in mind that the precise notation varies among statisticians: sometimes (as below) vectors are in square brackets and matrices in parentheses, but others mix and match otherwise, or always use one or the other. As long as it is clear in context, it’ll be fine. And if anyone gives you grief about your notation, just remind (or inform) them that notational conventions vary and ask them which part requires clarification.

$y_{i} \sim Normal(\mu_{i}, \sigma)$

$\mu_{i} = \alpha_{GROUP[i]} + \beta_{GROUP[i]} x_{i}$

$\begin{bmatrix} \alpha_{GROUP} \\ \beta_{GROUP} \end{bmatrix} \sim MvNormal(\begin{bmatrix} \alpha \\ \beta \end{bmatrix}, S)$

$S = \begin{pmatrix} \sigma_{\alpha} & 0 \\ 0 & \sigma_{\beta} \end{pmatrix}
     R
     \begin{pmatrix} \sigma_{\alpha} & 0 \\ 0 & \sigma_{\beta} \end{pmatrix}$

$\alpha \sim Normal(0, 10)$

$\beta \sim Normal(0, 10)$

$\sigma_{\alpha} \sim HalfCauchy(1)$

$\sigma_{\beta} \sim HalfCauchy(1)$

$\sigma \sim HalfCauchy(1)$

$R \sim LKJCorr(2)$

If you used different priors for $\beta$ and $\sigma_{\beta}$ and $R$, that’s fine. Just be sure your choices make sense to you.